# Seeded Poisson Factorization (SPF) example notebook

This notebook is a minimal example of the SPF model.

In [1]:
# Imports
from SPF.SPF_model import SPF
import numpy as np
import pandas as pd

df1 = pd.read_csv("./data/10k_amazon.csv")
df1.head()

Unnamed: 0   productId                                              Title  \
0       15123  B00032G1S0             Tuscan Whole Milk, 1 Gallon, 128 fl oz   
1       29859  B000O1LEK0         Kid Galaxy Mega X Morphibian Komodo dragon   
2       25129  B000G7WU50  Boon Flo Water Deflector and Protective Faucet...   
3       20101  B000K50KVO  Playtex Baby Einstein 6oz. Spill-Proof Trainer...   
4       18841  B0001E8BJ2  Pre~Seed Intimate Moisturizer with 2 Early Pre...   

           userId Helpfulness  Score        Time  \
0  A3B4TGHT06VIV9       19/20    5.0  1220659200   
1  A29BL1ME0A4S0U         1/1    4.0  1306800000   
2  A1M2CJJ859SZV7         1/1    2.0  1285372800   
3  A1YU39HHJTF0F0         0/0    1.0  1254700800   
4  A17YQZH6BBY9O4         2/2    5.0  1244851200   

                                                Text                  Cat1  \
0  It is a Tuscan Milkman,And he stoppeth only me...  grocery gourmet food   
1  This is a fun car. The only problem is the sho...            toys games   
2  This does indeed deflect the water for a softe...         baby products   
3  because my little girl loves to chew on her si...         baby products   
4  We got pregnant after using this product only ...  health personal care   

                   Cat2                            Cat3  tokens  
0            dairy eggs                            milk     106  
1  electronics for kids                         unknown      52  
2                safety                 bathroom safety     284  
3               feeding                   solid feeding      46  
4           health care  family planning contraceptives      49

In [2]:
# Define keywords
pets = ["dog", "cat", "cats", "dogs", "box", "food", "toy", "pet"]
toys = ["toy", "game", "play", "fun", "son", "daughter", "kids", "playing", "christmas", "toys", "child", "gift"]
beauty = ["skin", "color", "scent", "smell", "dry", "face", "look", "fragrance", "products", "dry", "perfume", "shampoo"]
baby = ["baby", "seat", "son", "daughter", "newborn", "months", "diaper", "diapers", "car", "stroller", "pump", "bag", "child"]
health = ["try", "long", "water", "feel", "shave", "razor", "shaver", "pain", "heart", "tooth", "balm", "pads", "taste", "drink","medical", "blade", "oil", "plastic", "shaker", "soap"]
grocery = ["tea", "taste", "flavor", "coffee", "chocolate", "sugar", "milk", "delicious", "water", "diet"]

keywords = {"pet supplies": pets, "toys games": toys, "beauty": beauty, "baby products": baby, "health personal care": health, "grocery gourmet food": grocery}

## SPF model

In [3]:
# Initialize the model
spf1 = SPF(keywords = keywords, residual_topics=0)
spf1

Seeded Poisson Factorization (SPF) model initialized with 6 keyword topics and 0 residual topics.

In [4]:
# Read documents and create the data required in the backend
spf1.read_docs(df1["Text"])

In [5]:
# Train the model
spf1.model_train(lr = 0.1, epochs = 150, tensorboard = False)

Instructions for updating:
The `interpolate_nondiscrete` flag is deprecated; instead use `force_probs_to_zero_outside_support` (with the opposite sense).


In [6]:
# See model results
spf1.plot_model_loss(neg_elbo = True)

In [7]:
betas = spf1.calculate_topic_word_distributions()
betas

pet supplies  toys games    beauty  baby products  \
00            1.555355    0.030551  0.028211       0.017095   
000           0.134427    0.045114  0.014117       0.065230   
02            0.009995    0.009615  0.006405       0.059267   
03            0.055968    0.008152  0.018273       0.011179   
04            0.018588    0.008089  0.027491       0.047931   
...                ...         ...       ...            ...   
zoo           0.013356    0.015119  0.007078       0.078367   
zoom          0.012518    0.061374  0.011115       0.013782   
zords         0.009773    0.043652  0.010369       0.008746   
zucchini      0.008882    0.006728  0.007080       0.008208   
zum           0.009388    0.009786  0.232785       0.008302   

          health personal care  grocery gourmet food  
00                    0.122050              0.262581  
000                   0.052856              0.181201  
02                    0.010808              0.010273  
03                    0.014627              0.021350  
04                    0.019409              0.013931  
...                        ...                   ...  
zoo                   0.011284              0.009590  
zoom                  0.008812              0.010105  
zords                 0.007993              0.010710  
zucchini              0.007228              0.049973  
zum                   0.008570              0.021951  

[13636 rows x 6 columns]

In [8]:
# Print the words with the highest mean intensity per topic
spf1.print_topics(num_words = 50)

array(['pet supplies: dog, product, box, amazon, just, cat, like, time, got, cats, price, dogs, great, bought, buy, don, food, did, small, litter, purchased, item, good, order, ordered, didn, little, money, work, plastic, shipping, pet, company, thing, use, size, works, thought, store, new, does, large, days, day, really, quality, received, away, came, sure',
       'toys games: toy, old, game, play, year, fun, like, loves, great, little, son, just, daughter, kids, really, set, time, got, toys, bought, christmas, think, playing, love, pieces, make, child, don, good, children, gift, doll, durable, music, buy, thing, played, cards, cute, birthday, recommend, easy, games, loved, cool, different, figure, things, big, come',
       'beauty: hair, product, like, skin, use, used, great, just, using, smell, dry, love, really, color, ve, does, time, products, good, little, long, face, don, doesn, years, scent, bottle, tried, works, day, look, soft, recommend, smells, soap, nice, feel, better, m

In [9]:
# Calculate model accuracy
categories, E_theta = spf1.calculate_topics()
df1["SPF_estimates"] = categories
df1["Accuracy"] = df1.Cat1 == df1.SPF_estimates

from sklearn.metrics import classification_report, confusion_matrix
import pprint
pprint.pprint(classification_report(df1.Cat1, df1.SPF_estimates))


('                      precision    recall  f1-score   support\n'
 '\n'
 '       baby products       0.68      0.78      0.73      1501\n'
 '              beauty       0.70      0.78      0.74      1374\n'
 'grocery gourmet food       0.63      0.89      0.73       855\n'
 'health personal care       0.78      0.48      0.60      2314\n'
 '        pet supplies       0.55      0.75      0.64      1253\n'
 '          toys games       0.91      0.81      0.86      2703\n'
 '\n'
 '            accuracy                           0.72     10000\n'
 '           macro avg       0.71      0.75      0.71     10000\n'
 '        weighted avg       0.75      0.72      0.72     10000\n')
